In [1]:
import tensorflow as tf

#张量概念是矢量概念的推广，矢量是一阶张量。张量是一个可用来表示在一些矢量、
#标量和其他张量之间的线性关系的多线性函数(可以理解成是向量、矩阵以及更高维结构的统称)。


from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
tf.reset_default_graph()

sess = tf.InteractiveSession()

x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])


def weight_variable(shape):
#从截断的正态分布中输出随机值，生成的值服从具有指定平均值和标准偏差的正态分布，如果生成的值大于平均值2个标准偏差的值则丢弃重新选择
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
#一个类，初始化器，它生成具有常量值的张量。由新张量的期望shape后面的参数value指定。参数value可以是常量值，
#也可以是类型为dtype的值列表。如果value是一个列表，那么列表的长度必须小于或等于由张量的期望形状所暗示的元素的数量。
#如果值中的元素总数小于张量形状所需的元素数，则值中的最后一个元素将用于填充剩余的元素。如果值中元素的总数大于张量形状所需元素的总数，
#初始化器将产生一个ValueError。

    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
#计算给定4-D输入和滤波器张量的2-D卷积.

"""
将滤镜展平为具有形状[filter_height * filter_width * in_channels, output_channels]的二维矩阵.
从输入张量中提取图像补丁,以形成形状为[batch, out_height, out_width, filter_height * filter_width * in_channels]的虚拟张量.
对于每个补丁,右对乘滤波器矩阵和图像补丁矢量.
"""
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")


def max_pool_2x2(x):

"""
max pooling是CNN当中的最大值池化操作，其实用法和卷积很类似
返回一个Tensor，类型不变，shape仍然是[batch, height, width, channels]这种形式

"""
    return tf.nn.max_pool(x, ksize=[1,2,2,1],
                          strides=[1,2,2,1], padding="SAME")


W_conv1 = weight_variable([5,5,1,32])
b_conv1 = bias_variable([32])

x_image = tf.reshape(x, [-1,28,28,1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([5,5,32,64])
b_conv2 = weight_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([7*7*64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

cross_entropy = -tf.reduce_sum(y_ * tf.log(y_conv))

train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_predict = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))

accuracy = tf.reduce_mean(tf.cast(correct_predict, "float"))

tf.summary.scalar('cross_entropy', cross_entropy)#
tf.summary.scalar('accuracy', accuracy)

sess.run(tf.initialize_all_variables())
#tfboard injection
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter("tblogs", sess.graph)

for i in range(200):
    batch = mnist.train.next_batch(50)
    if i%100 == 0:
        train_accuracy = accuracy.eval(feed_dict={
            x:batch[0], y_:batch[1], keep_prob:1.0})
        print ("step %d, training accuracy %g" % (i, train_accuracy))
        summary,_ = sess.run([merged,train_step],feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})
        writer.add_summary(summary,i)
    else:
        train_step.run(feed_dict={x:batch[0], y_:batch[1], keep_prob:0.5})

writer.flush()
print ("test accuracy %g" % accuracy.eval(feed_dict={x:mnist.test.images, y_:mnist.test.labels, keep_prob:1.0}))


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0, training accuracy 0.14
step 100, training accuracy 0.8
test accuracy 0.9115
